### EDA: Trực quan hóa và phân phối điểm theo môn, tổ hợp và năm

# Theo Môn

In [ ]:
# Xử lý data theo môn học

# Theo Tổ Hợp

In [ ]:
# Xử lý data theo tổ hợp khối

# Theo Năm 

In [ ]:
# Xử lý data qua từng năm